In [1]:
from common import *

cursor = connect()

# Current date and time

### current date

In [5]:
sql = """
SELECT CURRENT_DATE;
"""
run_sql(cursor, sql)

  current_date
0   2024-05-07


In [6]:
sql = """
SELECT 
  * 
FROM 
  rental 
WHERE 
  rental_date = CURRENT_DATE;
"""
run_sql(cursor, sql)

Empty DataFrame
Columns: [rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update]
Index: []


In [7]:
sql = """
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    date_of_birth DATE NOT NULL
);

INSERT INTO employees (name, date_of_birth) 
VALUES
    ('John Doe', '1992-05-15'),
    ('Jane Smith', '1995-08-22'),
    ('Bob Johnson', '1998-11-10')
RETURNING *;
"""
cursor.execute(sql)

ProgrammingError: the last operation didn't produce a result

In [8]:
sql = """
SELECT 
  name, 
  date_of_birth, 
  CURRENT_DATE as today,
  (CURRENT_DATE - date_of_birth) / 365 AS age 
FROM 
  employees 
ORDER BY 
  name;
"""
run_sql(cursor, sql)

          name date_of_birth       today  age
0  Bob Johnson    1998-11-10  2024-05-07   25
1   Jane Smith    1995-08-22  2024-05-07   28
2     John Doe    1992-05-15  2024-05-07   32


In [12]:
sql = """
CREATE TABLE delivery(
  delivery_id SERIAL PRIMARY KEY, 
  product VARCHAR(255) NOT NULL, 
  delivery_date DATE DEFAULT CURRENT_DATE
);

INSERT INTO delivery(product) 
VALUES 
  ('Sample screen protector');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x19d1265eda0>

In [13]:
sql = """
SELECT * FROM delivery;
"""
run_sql(cursor, sql)

   delivery_id                  product delivery_date
0            1  Sample screen protector    2024-05-07


### current time

In [18]:
sql = """
SELECT CURRENT_TIME;
"""
run_sql(cursor, sql)

            current_time
0  07:47:18.165315+08:00


In [29]:
sql = """
SELECT CURRENT_TIME(2);
"""
run_sql(cursor, sql)

            current_time
0  07:47:18.170000+08:00


In [30]:
sql = """
CREATE TABLE log (
    id SERIAL PRIMARY KEY,
    message VARCHAR(255) NOT NULL,
    created_at TIME DEFAULT CURRENT_TIME,
    created_on DATE DEFAULT CURRENT_DATE
);

INSERT INTO log( message )
VALUES('Testing the CURRENT_TIME function');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x19d1265f060>

In [31]:
sql = """
SELECT * FROM log;
"""
run_sql(cursor, sql)

   id                            message       created_at  created_on
0   1  Testing the CURRENT_TIME function  07:47:18.165315  2024-05-07


### current timestamp

In [32]:
sql = """
SELECT CURRENT_TIMESTAMP;
"""
run_sql(cursor, sql)

                 current_timestamp
0 2024-05-07 07:47:18.165315+08:00


In [33]:
sql = """
CREATE TABLE note (
    id SERIAL PRIMARY KEY,
    message VARCHAR(255) NOT NULL,
    created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
);

INSERT INTO note(message) 
VALUES('Testing current_timestamp function');
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x19d1265f060>

In [34]:
sql = """
SELECT * FROM note;
"""
run_sql(cursor, sql)

   id                             message                       created_at
0   1  Testing current_timestamp function 2024-05-07 07:47:18.165315+08:00


### clock timestamp

returns the current date and time with a timezone.

In [35]:
sql = """
SELECT CLOCK_TIMESTAMP();
"""
run_sql(cursor, sql)

                   clock_timestamp
0 2024-05-07 07:50:02.469037+08:00


In [36]:
sql = """
SELECT 
  clock_timestamp(),
  pg_sleep(3),
  clock_timestamp(),
  pg_sleep(3),
  clock_timestamp();
"""
run_sql(cursor, sql)

                   clock_timestamp pg_sleep                  clock_timestamp  \
0 2024-05-07 07:50:21.314437+08:00          2024-05-07 07:50:24.326485+08:00   

  pg_sleep                  clock_timestamp  
0          2024-05-07 07:50:27.329567+08:00  


In [37]:
sql = """
CREATE OR REPLACE FUNCTION time_it(
    p_statement TEXT
) RETURNS NUMERIC AS $$
DECLARE
    start_time TIMESTAMP WITH TIME ZONE;
    end_time TIMESTAMP WITH TIME ZONE;
    execution_time NUMERIC; -- ms
BEGIN
    -- Capture start time
    start_time := CLOCK_TIMESTAMP();

    -- Execute the statement
    EXECUTE p_statement;

    -- Capture end time
    end_time := CLOCK_TIMESTAMP();

    -- Calculate execution time in milliseconds
    execution_time := EXTRACT(EPOCH FROM end_time - start_time) * 1000;

    RETURN execution_time;
END;
$$ LANGUAGE plpgsql;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x19d1265f060>

In [38]:
sql = """
SELECT time_it('SELECT pg_sleep(3)');
"""
run_sql(cursor, sql)

       time_it
0  3014.853000


### statement timestamp

returns the start time of the current statement.

In [39]:
sql = """
SELECT STATEMENT_TIMESTAMP();
"""
run_sql(cursor, sql)

               statement_timestamp
0 2024-05-07 07:51:56.826056+08:00


In [4]:
sql = """
-- create a new table for logging
CREATE TABLE IF NOT EXISTS logs(
   id SERIAL PRIMARY KEY,
   started_at TIMESTAMP WITH TIME ZONE
);

-- start a transaction
BEGIN;
INSERT INTO logs(started_at) VALUES(statement_timestamp());
SELECT pg_sleep(3);

INSERT INTO logs(started_at) VALUES(statement_timestamp());
SELECT pg_sleep(3);

INSERT INTO logs(started_at) VALUES(statement_timestamp());
END;
"""
cursor.execute(sql)

关系 "logs" 已经存在, 跳过
已经有一个事物在运行中


<psycopg.Cursor [COMMAND_OK] [IDLE] (host=localhost port=15432 user=postgres database=dvdrental) at 0x21faf675850>

In [5]:
sql = """
-- retrieve data from the logs table
SELECT * FROM logs;
"""
run_sql(cursor, sql)

   id                       started_at
0   1 2024-05-07 07:53:09.627680+08:00
1   2 2024-05-07 07:53:09.627680+08:00
2   3 2024-05-07 07:53:09.627680+08:00
3   4 2024-05-07 07:55:45.613073+08:00
4   5 2024-05-07 07:55:45.613073+08:00
5   6 2024-05-07 07:55:45.613073+08:00
6   7 2024-05-07 07:56:51.923435+08:00
7   8 2024-05-07 07:56:51.923435+08:00
8   9 2024-05-07 07:56:51.923435+08:00


In [6]:
sql = """
SELECT
  statement_timestamp(), 
  pg_sleep(3), 
  statement_timestamp(), 
  pg_sleep(3), 
  statement_timestamp();
"""
run_sql(cursor, sql)

               statement_timestamp pg_sleep              statement_timestamp  \
0 2024-05-07 07:57:29.234642+08:00          2024-05-07 07:57:29.234642+08:00   

  pg_sleep              statement_timestamp  
0          2024-05-07 07:57:29.234642+08:00  


### now

returns the current date and time with the time zone of the database server.
请注意，NOW() 及其相关函数返回的是当前事务的开始时间。换句话说，在一个事务中，函数调用的返回值是相同的。

In [8]:
sql = """
SELECT NOW();
"""
run_sql(cursor, sql)

                               now
0 2024-05-07 07:57:00.588662+08:00


In [10]:
sql = """
SET TIMEZONE='Africa/Cairo';
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x21faf675850>

In [11]:
sql = """
SELECT NOW();
"""
run_sql(cursor, sql)

                               now
0 2024-05-07 02:57:00.588662+03:00


In [12]:
sql = """
SELECT * FROM pg_timezone_names;
"""
run_sql(cursor, sql)

                   name abbrev        utc_offset  is_dst
0        Africa/Abidjan    GMT   0 days 00:00:00   False
1          Africa/Accra    GMT   0 days 00:00:00   False
2    Africa/Addis_Ababa    EAT   0 days 03:00:00   False
3        Africa/Algiers    CET   0 days 01:00:00   False
4         Africa/Asmara    EAT   0 days 03:00:00   False
..                  ...    ...               ...     ...
592            US/Samoa    SST -1 days +13:00:00   False
593                 UTC    UTC   0 days 00:00:00   False
594                W-SU    MSK   0 days 03:00:00   False
595                 WET   WEST   0 days 01:00:00    True
596                Zulu    UTC   0 days 00:00:00   False

[597 rows x 4 columns]


In [13]:
sql = """
SELECT NOW()::timestamp;
"""
run_sql(cursor, sql)

                         now
0 2024-05-07 02:57:00.588662


In [14]:
sql = """
SELECT (NOW() + interval '1 day') AS this_time_tomorrow;
"""
run_sql(cursor, sql)

                this_time_tomorrow
0 2024-05-08 02:57:00.588662+03:00


In [15]:
sql = """
SELECT now() - interval '2 hours 30 minutes' AS two_hour_30_min_go;
"""
run_sql(cursor, sql)

                two_hour_30_min_go
0 2024-05-07 00:27:00.588662+03:00


In [16]:
sql = """
SELECT LOCALTIME, LOCALTIMESTAMP;
"""
run_sql(cursor, sql)

         localtime             localtimestamp
0  02:57:00.588662 2024-05-07 02:57:00.588662


In [17]:
sql = """
SELECT 
    TIMEOFDAY(), 
    pg_sleep(5), 
    TIMEOFDAY();
"""
run_sql(cursor, sql)

                              timeofday pg_sleep  \
0  Tue May 07 03:02:56.786439 2024 EEST            

                              timeofday  
0  Tue May 07 03:03:01.800025 2024 EEST  


### localtime and localtimestamp

returns the current time at which the current transaction starts.

In [18]:
sql = """
SELECT LOCALTIME;
"""
run_sql(cursor, sql)

         localtime
0  02:57:00.588662


In [19]:
sql = """
SELECT LOCALTIME(2);
"""
run_sql(cursor, sql)

         localtime
0  02:57:00.590000


In [21]:
sql = """
SELECT LOCALTIMESTAMP;
"""
run_sql(cursor, sql)

              localtimestamp
0 2024-05-07 02:57:00.588662


In [24]:
sql = """
SELECT LOCALTIMESTAMP(2);
"""
run_sql(cursor, sql)

           localtimestamp
0 2024-05-07 02:57:00.590


## Extracting date and time components

### date part

从日期或时间值中检索年、月、周等子字段。

In [25]:
sql = """
SELECT date_part('century',TIMESTAMP '2017-01-01');
"""
run_sql(cursor, sql)

   date_part
0       21.0


In [26]:
sql = """
SELECT date_part('year',TIMESTAMP '2017-01-01');
"""
run_sql(cursor, sql)

   date_part
0     2017.0


In [27]:
sql = """
SELECT date_part('quarter',TIMESTAMP '2017-01-01');
"""
run_sql(cursor, sql)

   date_part
0        1.0


In [28]:
sql = """
SELECT date_part('hour',TIMESTAMP '2017-03-18 10:20:30') h,
       date_part('minute',TIMESTAMP '2017-03-18 10:20:30') m,
       date_part('second',TIMESTAMP '2017-03-18 10:20:30') s;
"""
run_sql(cursor, sql)

      h     m     s
0  10.0  20.0  30.0


### extract

目的和 `date_part` 一样，语法不同。

In [29]:
sql = """
SELECT EXTRACT(YEAR FROM TIMESTAMP '2016-12-31 13:30:15') y;
"""
run_sql(cursor, sql)

      y
0  2016


## Converting to date and time

## Handling intervals

## Operators

## Utility functions